In [ ]:
!pip install tensorflow==2.16.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical


In [ ]:

data_text  = pd.read_csv('/content/df_commentary_new.csv')

In [ ]:
def data_processing(data,number_of_rows, list_of_columns):
    data = data.drop(columns= list_of_columns,axis=1)
    data = data.head(number_of_rows)
    return data

In [ ]:
def tokenize(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data['Modified_Commentary'])
    return tokenizer

In [ ]:
def input_sequences(data):
  input_sequences = []
  for i in range(len(data_text['Modified_Commentary'])):
      text = data_text['Modified_Commentary'][i]
      token_list = tokenizer.texts_to_sequences([text])[0]
      for i in range(1, len(token_list)):
          n_gram_sequence = token_list[:i+1]
          input_sequences.append(n_gram_sequence)

  max_len = max(len(x) for x in input_sequences)

  return input_sequences, max_len


In [23]:
max_len = max(len(x) for x in input_sequences)
max_len

153

In [ ]:
### Calling all the functions

dropped_columns = ['Unnamed: 0']
data_text =  data_processing(data_text, 1000, dropped_columns)
tokenizer  = tokenize(data_text)
input_sequences, max_len = input_sequences(data_text)
padded_input_sequences = pad_sequences(input_sequences, maxlen=max_len + 1, padding='pre')
padded_input_sequences
x = padded_input_sequences[:, :-1]
y = padded_input_sequences[:, -1]
y = to_categorical(y, num_classes=len(tokenizer.word_index)+1)
print('x shape : ',{x.shape})
print('x shape : ',{y.shape})


x shape :  {(45086, 153)}
x shape :  {(45086, 3583)}


In [ ]:

# print maximum word index

max_word_index = max([index for word, index in tokenizer.word_index.items()])
print(f"Maximum word index: {max_word_index}")

# print max length each string

max_length = data_text['Modified_Commentary'].str.len().max()

print(f"Maximum length of a string: {max_length}")

max_text_index = data_text[data_text['Modified_Commentary'].str.len() == max_length].index[0]

print(f"Index of maximum length of a string: {max_text_index}")


Maximum word index: 3582
Maximum length of a string: 798
Index of maximum length of a string: 325


In [ ]:
def rnn_model():

  model_rnn = Sequential()
  model_rnn.add(Embedding(input_dim=3583, output_dim= 100, input_length=max_len))
  model_rnn.add(SimpleRNN(100, return_sequences=True)) # Pass return_sequences to LSTM
  model_rnn.add(SimpleRNN(100))
  model_rnn.add(Dense(3583, activation='softmax'))
  model_rnn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  return model_rnn


In [ ]:
model = rnn_model()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_1 (SimpleRNN)             │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(x,y,epochs= 50)

Epoch 1/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 196s 134ms/step - accuracy: 0.0669 - loss: 6.3407
Epoch 2/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 213s 143ms/step - accuracy: 0.1078 - loss: 5.5329
Epoch 3/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 201s 143ms/step - accuracy: 0.1653 - loss: 4.9323
Epoch 4/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 200s 142ms/step - accuracy: 0.1993 - loss: 4.5457
Epoch 5/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 210s 149ms/step - accuracy: 0.2276 - loss: 4.2437
Epoch 6/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 185s 131ms/step - accuracy: 0.2467 - loss: 4.0219
Epoch 7/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 180s 128ms/step - accuracy: 0.2692 - loss: 3.7896
Epoch 8/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 204s 129ms/step - accuracy: 0.2880 - loss: 3.6104
Epoch 9/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 200s 127ms/step - accuracy: 0.3179 - loss: 3.4011
Epoch 10/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 179s 127ms/step - accuracy: 0.3333 - loss: 3.2322
Epoch 11/50
1409/1409 ━━━━━━━━━━━━━━━━━━━━ 199s 125ms/step - accuracy: 0.3572 -

## With accuracy of 50 percent

In [ ]:

eval_data = data_text.iloc[-5:]
eval_sequences = tokenizer.texts_to_sequences(eval_data['Modified_Commentary'])
max_predict_len = 10

# Function to predict the next n words
def predict_next_n_words(model, tokenizer, seed_text, n_words):
    result = []
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=1)[0]
        predicted_word = tokenizer.index_word[predicted_index]
        result.append(predicted_word)
        seed_text += ' ' + predicted_word
    return result

# Evaluate the model
reverse_word_index = dict((i, word) for word, i in tokenizer.word_index.items())
for i, seq in enumerate(eval_sequences):
    if len(seq) > 10:
        first_10_words = seq[:10]
    else:
        first_10_words = seq

    first_10_words_text = ' '.join(reverse_word_index.get(word, '?') for word in first_10_words)
    predicted_words = predict_next_n_words(model, tokenizer, first_10_words_text, max_predict_len)
    actual_next_10_words = seq[10:20] if len(seq) > 20 else seq[10:]
    actual_next_10_words_text = ' '.join(reverse_word_index.get(word, '?') for word in actual_next_10_words)

    print(f"Sentence {i+1}:")
    print(f"First 10 words: {first_10_words_text}")
    print(f"Actual next 10 words: {actual_next_10_words_text}")
    print(f"Predicted next 10 words: {' '.join(predicted_words)}")
    print()








Sentence 1:
First 10 words: out caught by uthappa umesh yadav strikes and removes the
Actual next 10 words: big fish he bowled much better compared to his first
Predicted next 10 words: big fish he bowled a little shimmy down the track

Sentence 2:
First 10 words: four that's harsh on the bowler that didn't deserve to
Actual next 10 words: go to the boundary it was banged in short and
Predicted next 10 words: go to the boundary mohit moves across and swishes it

Sentence 3:
First 10 words: four welcome to delhi says iyer shortish and wide outside
Actual next 10 words: off iyer treks back and fiercely cuts it behind square
Predicted next 10 words: off iyer treks back and fiercely cuts it behind square

Sentence 4:
First 10 words: out caught by billings zak attack strikes trap set and
Actual next 10 words: trap sprung so batsman's promotion fails lasted for just two
Predicted next 10 words: trap sprung so batsman's promotion fails lasted for a strategic

Sentence 5:
First 10 words: four 

In [ ]:
eval_data_1 = data_text.iloc[-10:]

eval_sequences = tokenizer.texts_to_sequences(eval_data_1['Modified_Commentary'])
max_predict_len = 10

# Function to predict the next n words
def predict_next_n_words(model, tokenizer, seed_text, n_words):
    result = []
    for _ in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_len, padding='pre')
        predicted_probs = model.predict(token_list, verbose=0)
        predicted_index = np.argmax(predicted_probs, axis=1)[0]
        predicted_word = tokenizer.index_word[predicted_index]
        result.append(predicted_word)
        seed_text += ' ' + predicted_word
    return result

# Evaluate the model
reverse_word_index = dict((i, word) for word, i in tokenizer.word_index.items())
for i, seq in enumerate(eval_sequences):
    if len(seq) > 10:
        first_10_words = seq[:10]
    else:
        first_10_words = seq

    first_10_words_text = ' '.join(reverse_word_index.get(word, '?') for word in first_10_words)
    predicted_words = predict_next_n_words(model, tokenizer, first_10_words_text, max_predict_len)
    actual_next_10_words = seq[10:20] if len(seq) > 20 else seq[10:]
    actual_next_10_words_text = ' '.join(reverse_word_index.get(word, '?') for word in actual_next_10_words)

    print(f"Sentence {i+1}:")
    print(f"First 10 words: {first_10_words_text}")
    print(f"Actual next 10 words: {actual_next_10_words_text}")
    print(f"Predicted next 10 words: {' '.join(predicted_words)}")
    print()








Sentence 1:
First 10 words: four exquisite he's seeing the cricket ball like a football
Actual next 10 words: all timing he is not trying to hit the leather
Predicted next 10 words: all the power to go past it past the diving

Sentence 2:
First 10 words: four starts with a full toss on leg easy peasy
Actual next 10 words: for batsman he clips it in front of square on
Predicted next 10 words: for batsman he clips it in front of square on

Sentence 3:
First 10 words: 2 runs to long on 50 run stand in the
Actual next 10 words: fifth over a gentle nudge into the leg side followed
Predicted next 10 words: fifth over a gentle nudge into the leg side followed

Sentence 4:
First 10 words: out caught by uthappa the umpire took ages to raise
Actual next 10 words: his finger perhaps it looked like he had a look
Predicted next 10 words: his finger perhaps it looked like he gets a faint

Sentence 5:
First 10 words: four cheeky delicate and effective uses the slightest of width
Actual next 10 words:

In [ ]:
model.fit(x,y,epochs=1)

1409/1409 ━━━━━━━━━━━━━━━━━━━━ 171s 121ms/step - accuracy: 0.7960 - loss: 0.7946


In [ ]:
from tensorflow.keras.models import load_model, save_model

In [ ]:
save_model(model, 'model_rnn_1.keras')


## With model accuracy of 90 percent

In [ ]:
model.fit(x,y,epochs=15)

NameError: name 'model' is not defined